# Project objective
In this project, we build a neural network model for predicting tissue of origin of cancer cell lines using their gene expression provided in cancer cell line encyclopedia dataset. Although by definition the built model is a deep learning model, I try to avoid it for now.

Information about the dataset, some technical details about the used machine learning method(s) and mathematical details of the quantifications approaches are provided in the code. 

# Packages we work with in this notebook
We are going to use the following libraries and packages:

* **numpy**: NumPy is the fundamental package for scientific computing with Python. (http://www.numpy.org/)
* **sklearn**: Scikit-learn is a machine learning library for Python programming language. (https://scikit-learn.org/stable/)
* **pandas**: Pandas provides easy-to-use data structures and data analysis tools for Python. (https://pandas.pydata.org/)
* **keras**: keras is a widely-used neural network framework in python. 

In [19]:
import numpy as np
import pandas as pd
import keras 

# Introduction to the dataset

**Name**: Cancer Cell Line Encyclopedia dataset

**Summary**: Identifying tissue of origin of cancer cell lines using their gene expression. Cell lines from 6 tissues were chosen for this code including: breast, central_nervous_system, haematopoietic_and_lymphoid_tissue, large_intestine, lung, skin

**number of features**: 500 (real, positive) 
Top 500 genex based on variance of their expression in the dataset is chosen. The right way to select the features is to do it only on the training set to eliminate information leak from test set. But to simplify the process for the sake of this teaching code, we use all the dataset.

**Number of data points (instances)**: 550

**dataset accessibility**: Dataset is available as part of PharmacoGx R package (https://www.bioconductor.org/packages/release/bioc/html/PharmacoGx.html)

**Link to the dataset**: https://portals.broadinstitute.org/ccle




## Importing the dataset
We can import the dataset in multiple ways

**Colab Notebook**: You can download the dataset file (or files) from the link (if provided) and uploading it to your google drive and then you can import the file (or files) as follows:

**Note.** When you run the following cell, it tries to connect the colab with google derive. Follow steps 1 to 5 in this link (https://www.marktechpost.com/2019/06/07/how-to-connect-google-colab-with-google-drive/) to complete the 

In [20]:
from google.colab import drive
drive.mount('/content/gdrive')

# This path is common for everybody
# This is the path to your google drive
input_path = '/content/gdrive/My Drive/'
# reading the data (target)
target_dataset_features = pd.read_csv(input_path + 'CCLE_ExpMat_Top500Genes.csv', index_col=0)
target_dataset_output = pd.read_csv(input_path + 'CCLE_ExpMat_Phenotype.csv', index_col=0)
# Transposing the dataframe to put features in the dataframe columns
target_dataset_features = target_dataset_features.transpose()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Local directory**: In case you save the data in your local directory, you need to change "input_path" to the local directory you saved the file (or files) in.

**GitHub**: If you use my GitHub (or your own GitHub) repo, you need to change the "input_path" to where the file (or files) exist in the repo. For example, when I clone ***ml_in_practice*** from my GitHub, I need to change "input_path" to 'data/' as the file (or files) is saved in the data dicretory in this repository. 

**Note.**: You can also clone my ***ml_in_practice*** repository (here: https://github.com/alimadani/ml_in_practice) and follow the same process.

## Making sure about the dataset characteristics (number of data points and features)

In [21]:
print('number of features: {}'.format(target_dataset_features.shape[1]))
print('number of data points: {}'.format(target_dataset_features.shape[0]))

number of features: 500
number of data points: 550


## Data preparation
We need to prepare the dataset for machine learnign modeling. Here we prepare the data in 2 steps:

1) Selecting target columns from the output dataframe (target_dataset_output)

2) Converting tissue names to integers (one for each tissue)

3) Converting the integer array of labels to one-hot encodings to be used in neural network modeling

In [22]:
# tissueid is the column that contains tissue type information
output_var_names = target_dataset_output['tissueid']
# converting tissue names to labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(output_var_names)
output_var = le.transform(output_var_names)

class_number = len(np.unique(output_var))
# transforming the output array to an array of one hot vectors
# it measn that we have a vector for each datapoint
# with length equal to the number of classes
# Depending on the class of each datapoint, 
# one of the values (for that class) will be one
# and the rest of them will be zero for each data point 

# .reshape(-1,1) has to be used to transform a 1d array of class
# numbers to a 2d array ready to be encoded by OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
output_var = ohe.fit_transform(output_var.reshape(-1,1)).toarray()

# we would like to use all the features as input features of the model
input_features = target_dataset_features

## Splitting data to training and testing sets

We need to split the data to train and test, if we do not have a separate dataset for validation and/or testing, to make sure about generalizability of the model we train.

**test_size**: Traditionally, 30%-40% of the dataset cna be used for test set. If you split the data to train, validation and test, you can use 60%, 20% and 20% of the dataset, respectively.

**Note.**: We need the validation and test sets to be big enough for checking generalizability of our model. At the same time we would like to have as much data as possible in the training set to train a better model.

**random_state** as the name suggests, is used for initializing the internal random number generator, which will decide the splitting of data into train and test indices in your case.


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_features, output_var, test_size=0.30, random_state=5)

## Building the supervised learning model
We want to build a multi-class classification model as the output variable include multiple classes. Here we build a neural network model with 2 hidden layers. A neural network with 2 or more hidden layers are called deep neural network. So technical it is a deep learning code. As you can see the implementation of a deep learning model is not difficult. But knowing how to interpret it, how to fine-tune the model and avoid overfitting are the parts that need experience and more knowledge.


### Fully connected neural network


In [24]:
from keras.models import Sequential
from keras.layers import Dense
# building a neural network model
model = Sequential()
# adding 1st hidden layer with 128 neurons and relu as its activation function
# input_dim should be specified as the number of input features
model.add(Dense(128, input_dim=target_dataset_features.shape[1], activation='relu'))
# adding 2nd hidden layer with 32 neurons and relu as its activation function
model.add(Dense(64, activation='relu'))
# adding the output layer (softmax is used to generate probabilities for each predicted class)
# Size if the last layer should be equal to the total number of classes in the dataset
model.add(Dense(class_number, activation='softmax'))

# compiling the model using cross-entropy for categorical variables,
# as we are dealing with multi-class classification
# Adam optimization algorithm is also used
# Accuracy is used as the metric to assess performance of our model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Now we fit our neural network model using the trainign set:

In [25]:
# Train the model using the training set
model.fit(X_train, y_train, epochs=300, batch_size=64)


Epoch 1/300
385/385 [==============================] - 0s 208us/step - loss: 2.6319 - accuracy: 0.4701
Epoch 2/300
385/385 [==============================] - 0s 59us/step - loss: 1.2702 - accuracy: 0.6052
Epoch 3/300
385/385 [==============================] - 0s 57us/step - loss: 0.8191 - accuracy: 0.7506
Epoch 4/300
385/385 [==============================] - 0s 46us/step - loss: 0.6767 - accuracy: 0.8000
Epoch 5/300
385/385 [==============================] - 0s 53us/step - loss: 0.7844 - accuracy: 0.7662
Epoch 6/300
385/385 [==============================] - 0s 47us/step - loss: 0.6166 - accuracy: 0.8286
Epoch 7/300
385/385 [==============================] - 0s 60us/step - loss: 0.4902 - accuracy: 0.8468
Epoch 8/300
385/385 [==============================] - 0s 45us/step - loss: 0.7123 - accuracy: 0.7818
Epoch 9/300
385/385 [==============================] - 0s 45us/step - loss: 0.4307 - accuracy: 0.8909
Epoch 10/300
385/385 [==============================] - 0s 52us/step - loss: 0.64

The model is trained now and can be used to predict the lables of datapoints in the test set.
Note. To be able to assess the performance of the predictions in the test set using metrics class in sklearn, we need to transform the true lables and the predictions from one-hot encodings to lists.

In [26]:
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

## Evaluating performance of the model
We need to assess performance of the model using the predictions of the test set. We use accuracy and balanced accuracy. Here are their definitions:

* **recall** in this context is also referred to as the true positive rate or sensitivity

How many relevant item are selected




$${\displaystyle {\text{recall}}={\frac {tp}{tp+fn}}\,} $$

 

* **specificity** true negative rate



$${\displaystyle {\text{true negative rate}}={\frac {tn}{tn+fp}}\,}$$

* **accuracy**: This measure gives you a sense of performance for all the classes together as follows:

$$ {\displaystyle {\text{accuracy}}={\frac {tp+tn}{tp+tn+fp+fn}}\,}$$


\begin{equation*} accuracy=\frac{number\:of\:correct\:predictions}{(total\:number\:of\:data\:points (samples))} \end{equation*}


* **balanced accuracy**: This measure gives you a sense of performance for all the classes together as follows:

$${\displaystyle {\text{balanced accuracy}}={\frac {recall+specificity
}{2}}\,}$$


In [27]:
from sklearn import metrics

print('Accuracy of the neural network model is:', metrics.accuracy_score(pred,test)*100)

print("Blanced accuracy of the neural network model is:", metrics.balanced_accuracy_score(pred, test))

Accuracy of the neural network model is: 92.72727272727272
Blanced accuracy of the neural network model is: 0.9156156880615085
